In [ ]:
#From SAV to LOVE

# HW7

### 1.Построить обобщенную линейную модель (GLM) для прогнозирования наступления страховых случаев на рассмотренных в ноутбуке данных.

### 2.Подобрать необходимое распределение и тип связи, при необходимости ознакомиться с документацией H2O.

### 3. Придумать и использовать дополнительные факторы при построении модели (например, пересечения признаков или функции от них и т.д.).

### 4. Оценить результаты построенной модели при помощи различных метрик (можно использовать и другие метрики помимо представленных в ноутбуке), проанализировать вероятные проблемы.

### 5.Предложить способы их решения и/или попробовать их решить, улучшив результат

In [1]:
# Подключение к Google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
PATH ='/content/drive/My Drive/gb/' 

In [4]:
# Загрузим набор данных

df = pd.read_csv('freMPL-R.csv', low_memory=False)
df = df.loc[df.Dataset.isin([5, 6, 7, 8, 9])]
df.drop('Dataset', axis=1, inplace=True)
df.dropna(axis=1, how='all', inplace=True)
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)

In [6]:
NegClaimAmount = df.loc[df.ClaimAmount < 0, ['ClaimAmount','ClaimInd']]
print('Unique values of ClaimInd:', NegClaimAmount.ClaimInd.unique())
NegClaimAmount.head()

Unique values of ClaimInd: [0]


,ClaimAmount,ClaimInd
82,-74.206042,0
175,-1222.585196,0
177,-316.288822,0
363,-666.758610,0
375,-1201.600604,0


In [8]:
df.loc[df.ClaimAmount < 0, 'ClaimAmount'] = 0

In [9]:
df['ClaimsCount'] = df.ClaimInd + df.ClaimNbResp + df.ClaimNbNonResp + df.ClaimNbParking + df.ClaimNbFireTheft + df.ClaimNbWindscreen
df.loc[df.ClaimAmount == 0, 'ClaimsCount'] = 0
df.drop(["ClaimNbResp", "ClaimNbNonResp", "ClaimNbParking", "ClaimNbFireTheft", "ClaimNbWindscreen"], axis=1, inplace=True)

In [10]:
pd.DataFrame(df.ClaimsCount.value_counts()).rename({'ClaimsCount': 'Policies'}, axis=1)

,Policies
0.0,104286
2.0,3529
1.0,3339
3.0,2310
4.0,1101
5.0,428
6.0,127
7.0,26
8.0,6
9.0,2


In [11]:
df.loc[df.ClaimsCount > 0, 'AvgClaim'] = df['ClaimAmount']/df['ClaimsCount']

In [12]:
class InsDataFrame:


    ''' Load data method '''

    def load_pd(self, pd_dataframe):
        self._df = pd_dataframe


    ''' Columns match method '''

    def columns_match(self, match_from_to):
        self._df.rename(columns=match_from_to, inplace=True)


    ''' Person data methods '''

    # Gender
    _gender_dict = {'Male':0, 'Female':1}

    def transform_gender(self):
        self._df['Gender'] = self._df['Gender'].map(self._gender_dict)

        

    # Age
    @staticmethod
    def _age(age, age_max):
        if pd.isnull(age):
            age = None
        elif age < 18:
            age = None
        elif age > age_max:
            age = age_max
        return age
      
    def transform_age(self, age_max=70):
        self._df['driver_minage'] = self._df['driver_minage'].apply(self._age, args=(age_max,))

    # Age M/F
    @staticmethod
    def _age_gender(age_gender):
        _age = age_gender[0]
        _gender = age_gender[1]
        if _gender == 0: #Male
            _driver_minage_m = _age
            _driver_minage_f = 18
        elif _gender == 1: #Female
            _driver_minage_m = 18
            _driver_minage_f = _age
        else:
            _driver_minage_m = 18
            _driver_minage_f = 18
        return [_driver_minage_m, _driver_minage_f]
    
    def transform_age_gender(self):
        self._df['driver_minage_m'],self._df['driver_minage_f'] = zip(*self._df[['driver_minage','Gender']].apply(self._age_gender, axis=1).to_frame()[0])

    # Experience
    @staticmethod
    def _exp(exp, exp_max):
        if pd.isnull(exp):
            exp = None
        elif exp < 0:
            exp = None
        elif exp > exp_max:
            exp = exp_max
        return exp

    def transform_exp(self, exp_max=52):
        self._df['driver_minexp'] = self._df['driver_minexp'].apply(self._exp, args=(exp_max,))


    ''' Other data methods '''

    def polynomizer(self, column, n=2):
        if column in list(self._df.columns):
            for i in range(2,n+1):
                self._df[column+'_'+str(i)] = self._df[column]**i

    def get_dummies(self, columns):
        self._df = pd.get_dummies(self._df, columns=columns)


    ''' General methods '''

    def info(self):
        return self._df.info()

    def head(self, columns, n=5):
        return self._df.head(n)

    def len(self):
        return len(self._df)

    def get_pd(self, columns):
        return self._df[columns]

In [13]:
class InsDataFrame_Fr(InsDataFrame):

    # Experience (weeks to years)
    @staticmethod
    def _exp(exp, exp_max):
        if pd.isnull(exp):
            exp = None
        elif exp < 0:
            exp = None
        else:
            exp * 7 // 365
        if exp > exp_max:
            exp = exp_max
        return exp

    # Marital status
    _MariStat_dict = {'Other':0, 'Alone':1}

    def transform_MariStat(self):
        self._df['MariStat'] = self._df['MariStat'].map(self._MariStat_dict)
    
    # Social category
    def transform_SocioCateg(self):
        self._df['SocioCateg'] = self._df['SocioCateg'].str.slice(0,4)

In [14]:
data = InsDataFrame_Fr()

In [15]:
data.load_pd(df)

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115155 entries, 0 to 115154
Data columns (total 17 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Exposure     115155 non-null  float64
 1   LicAge       115155 non-null  int64  
 2   RecordBeg    115155 non-null  object 
 3   RecordEnd    59455 non-null   object 
 4   Gender       115155 non-null  object 
 5   MariStat     115155 non-null  object 
 6   SocioCateg   115155 non-null  object 
 7   VehUsage     115155 non-null  object 
 8   DrivAge      115155 non-null  int64  
 9   HasKmLimit   115155 non-null  int64  
 10  BonusMalus   115155 non-null  int64  
 11  ClaimAmount  115155 non-null  float64
 12  ClaimInd     115155 non-null  int64  
 13  OutUseNb     115155 non-null  float64
 14  RiskArea     115155 non-null  float64
 15  ClaimsCount  115155 non-null  float64
 16  AvgClaim     10869 non-null   float64
dtypes: float64(6), int64(5), object(6)
memory usage: 14.9+ MB


In [17]:
# Переименовываем
data.columns_match({'DrivAge':'driver_minage','LicAge':'driver_minexp'})

In [18]:
# Преобразовываем
data.transform_age()
data.transform_exp()
data.transform_gender()
data.transform_MariStat()
data.transform_SocioCateg()

In [19]:
# Пересечение пола и возраста, их квадраты
data.transform_age_gender()
data.polynomizer('driver_minage_m')
data.polynomizer('driver_minage_f')

In [20]:
# Onehot encoding
data.get_dummies(['VehUsage','SocioCateg'])

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115155 entries, 0 to 115154
Data columns (total 30 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   Exposure                         115155 non-null  float64
 1   driver_minexp                    115155 non-null  int64  
 2   RecordBeg                        115155 non-null  object 
 3   RecordEnd                        59455 non-null   object 
 4   Gender                           115155 non-null  int64  
 5   MariStat                         115155 non-null  int64  
 6   driver_minage                    115155 non-null  int64  
 7   HasKmLimit                       115155 non-null  int64  
 8   BonusMalus                       115155 non-null  int64  
 9   ClaimAmount                      115155 non-null  float64
 10  ClaimInd                         115155 non-null  int64  
 11  OutUseNb                         115155 non-null  float64
 12  Ri

In [23]:
col_features = [
                'driver_minexp',
                'Gender',
                'MariStat',
                'HasKmLimit',
                'BonusMalus',
                'OutUseNb',
                'RiskArea',
                'driver_minage_m',
                'driver_minage_f',
                'driver_minage_m_2',
                'driver_minage_f_2',
                'VehUsage_Private',
                'VehUsage_Private+trip to office',
                'VehUsage_Professional',
                'VehUsage_Professional run',
                'SocioCateg_CSP1',
                'SocioCateg_CSP2',
                'SocioCateg_CSP3',
                'SocioCateg_CSP4',
                'SocioCateg_CSP5',
                'SocioCateg_CSP6',
                'SocioCateg_CSP7'  
]

In [24]:
col_target = ['ClaimAmount', 'ClaimsCount', 'AvgClaim']

In [25]:
df_freq = data.get_pd(col_features+col_target)

In [26]:
df_ac = df_freq[df_freq['ClaimsCount'] > 0].reset_index().copy()

In [28]:
!java -version

openjdk version "11.0.9.1" 2020-11-04
OpenJDK Runtime Environment (build 11.0.9.1+1-Ubuntu-0ubuntu1.18.04)
OpenJDK 64-Bit Server VM (build 11.0.9.1+1-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)


In [30]:
import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.9.1" 2020-11-04; OpenJDK Runtime Environment (build 11.0.9.1+1-Ubuntu-0ubuntu1.18.04); OpenJDK 64-Bit Server VM (build 11.0.9.1+1-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpv3yd3h3e
  JVM stdout: /tmp/tmpv3yd3h3e/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpv3yd3h3e/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.3
H2O_cluster_version_age:,1 month and 23 days
H2O_cluster_name:,H2O_from_python_unknownUser_7ps1rb
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.180 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


In [31]:
from sklearn.model_selection import train_test_split

In [43]:
# Разбиение датасета на train/val/test

df_ind = data.get_pd(col_features+['ClaimInd'])

x_train_ind, x_test_ind, y_train_ind, y_test_ind = train_test_split(df_ind[col_features], df_ind.ClaimInd, test_size=0.3, random_state=1)
x_valid_ind, x_test_ind, y_valid_ind, y_test_ind = train_test_split(x_test_ind, y_test_ind, test_size=0.5, random_state=1)

In [44]:
# Преобразование в H2O-Frame

# Преобразование в H2O-Frame

h2o_train_c = h2o.H2OFrame(pd.concat([x_train_ind, y_train_ind], axis=1))
h2o_valid_c = h2o.H2OFrame(pd.concat([x_valid_ind, y_valid_ind], axis=1))
h2o_test_c = h2o.H2OFrame(pd.concat([x_test_ind, y_test_ind], axis=1))

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [46]:
# Преобразуем целевую переменную ClaimInd в категориальную при помощи метода asfactor во всех наборах данных

h2o_train_c['ClaimInd'] = h2o_train_c['ClaimInd'].asfactor()
h2o_valid_c['ClaimInd'] = h2o_valid_c['ClaimInd'].asfactor()
h2o_test_c['ClaimInd'] = h2o_test_c['ClaimInd'].asfactor()

In [48]:
# Инициализируем и обучим GLM модель c кросс-валидацией

glm_bin = H2OGeneralizedLinearEstimator(family = "binomial", link = "logit", nfolds=5, balance_classes=True)
glm_bin.train(y='ClaimInd', x = h2o_train_c.names[1:-1], training_frame = h2o_train_c, validation_frame = h2o_valid_c)

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [50]:
# Параметры модели: распределение, функция связи, гиперпараметры регуляризации, количество использованных объясняющих переменных

glm_bin.summary()


GLM Model: summary


,,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
0,,binomial,logit,"Elastic Net (alpha = 0.5, lambda = 2.368E-5 )",21,21,3,py_1_sid_8baa


In [52]:
# Метрики качества модели - по всем данным и на кросс-валидации

glm_bin.cross_validation_metrics_summary().as_data_frame()

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.53960186,0.12766887,0.3170807,0.63467413,0.58901715,0.6026652,0.5545721
1,auc,0.56284577,0.0052034375,0.56108105,0.5666864,0.5577072,0.5698242,0.55892986
2,aucpr,0.113302656,0.0047695185,0.11288304,0.11455467,0.10895557,0.12072462,0.10939537
3,err,0.46039814,0.12766887,0.68291926,0.36532584,0.41098285,0.39733484,0.4454279
4,err_count,7428.6,2090.28,11051.0,5858.0,6586.0,6351.0,7297.0
5,f0point5,0.13487232,0.0073751304,0.12572339,0.14219563,0.1300625,0.14237855,0.13400148
6,f1,0.18716738,0.0058622384,0.18412699,0.19155397,0.1794169,0.1941378,0.18660127
7,f2,0.3077199,0.021591706,0.34386718,0.29339647,0.2891334,0.3050239,0.3071785
8,lift_top_group,1.3339299,0.25843644,1.5025798,1.3044698,1.4170965,1.5438507,0.90165275
9,logloss,0.3112855,0.0037914102,0.31077084,0.31239107,0.30561787,0.31616944,0.31147826


In [53]:
# Таблица коэффициентов модели (в зависимости от модели могут выводиться также стандартная ошибка, z-score и p-value)
glm_bin._model_json['output']['coefficients_table'].as_data_frame()

,names,coefficients,standardized_coefficients
0,Intercept,-2.475348e+00,-2.279640
1,Gender,-6.416985e-02,-0.031103
2,MariStat,-6.469029e-02,-0.023304
3,HasKmLimit,-3.698560e-01,-0.115502
4,BonusMalus,6.792484e-03,0.104267
5,OutUseNb,6.145252e-02,0.042753
6,RiskArea,9.240660e-03,0.020474
7,driver_minage_m,-3.727295e-03,-0.070187
8,driver_minage_f,-1.168549e-03,-0.018743
9,driver_minage_m_2,-6.145260e-06,-0.009595


In [ ]:
# Таблица нормированных коэффициентов по всем данным и на кросс-валидации

pmodels = {}
pmodels['overall'] = glm_bin.coef_norm()
for x in range(len(glm_bin.cross_validation_models())):
    pmodels[x] = glm_bin.cross_validation_models()[x].coef_norm()
coef = pd.DataFrame.from_dict(pmodels).round(5)
coef['overall'] = abs(coef['overall'])
coef.sort_values('overall',ascending=False)

In [55]:
# Построение прогнозных значений для обучающей, валидационной и тестовой выборок

c_train_pred = glm_bin.predict(h2o_train_c).as_data_frame()
c_valid_pred = glm_bin.predict(h2o_valid_c).as_data_frame()
c_test_pred = glm_bin.predict(h2o_test_c).as_data_frame()

glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%


In [65]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

In [63]:
# Выведем импортированные выше метрики классификации для обучающей, валидационной и тестовой выборок
#default f1
glm_bin.confusion_matrix(train=True, valid=True)


Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.0948423489611114: 


,,0,1,Error,Rate
0,0,42781.0,30192.0,0.4137,(30192.0/72973.0)
1,1,3734.0,3901.0,0.4891,(3734.0/7635.0)
2,Total,46515.0,34093.0,0.4209,(33926.0/80608.0)



Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.09477299981562572: 


,,0,1,Error,Rate
0,0,9217.0,6447.0,0.4116,(6447.0/15664.0)
1,1,783.0,826.0,0.4866,(783.0/1609.0)
2,Total,10000.0,7273.0,0.4186,(7230.0/17273.0)


{'train': , 'valid': }

In [60]:
glm_bin.accuracy()

[[0.2855755587557923, 0.9052699483922191]]

In [69]:
print(confusion_matrix(c_test_pred['predict'],  y_test_ind))
print(f1_score(c_test_pred['predict'],  y_test_ind))
print(accuracy_score(c_test_pred['predict'],  y_test_ind))

[[9168  801]
 [6481  824]]
0.18454647256438972
0.5784415885145305


In [71]:
print(confusion_matrix(c_train_pred['predict'],  y_train_ind))
print(f1_score(c_train_pred['predict'],  y_train_ind))
print(accuracy_score(c_train_pred['predict'],  y_train_ind))

[[42814  3741]
 [30159  3894]]
0.18681635002878524
0.579446208813021


In [72]:
print(confusion_matrix(c_valid_pred['predict'],  y_valid_ind))
print(f1_score(c_valid_pred['predict'],  y_valid_ind))
print(accuracy_score(c_valid_pred['predict'],  y_valid_ind))

[[9240  784]
 [6424  825]]
0.18627229622939714
0.5827013257685405
